# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

import module as mod     # functions are include in module.py

'''pd.options.display.max_columns = None
pd.options.display.max_rows = None'''

'pd.options.display.max_columns = None\npd.options.display.max_rows = None'

Cuantas causas de muerte existen? '102 en 14 grupos'

Distribuición por edades? => Histograma

Cuál es la mayor causa de la muerte histórica?

Cuáles la evolución de muertes por ______?

Cuáles son las principales causas de muerte infantil y su evolución en el tiempo? 

Cuál es la causa de muerte que menos os debe preocupar?

Varias librerías; potly, cuf, sns, matplotlib

ETLs, KPIs

In [2]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
deaths = deaths.replace('95 y más años','De 95 y más años').replace('De 5 a 9 años','De 05 a 09 años').replace('Menos de 1 año','De 00 a 01 año').replace('De 1 a 4 años','De 01 a 04 años')


Bins

In [4]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


# Data 'Edad' --> put a zero to any, for sort by 'Edad'

In [5]:
deaths.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [6]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, De 00 a 01 año, De 01 a 04 ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [7]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table.iloc[63]

cause_code                                          056
cause_name    Otras enfermedades isquémicas del corazón
Name: 63, dtype: object

In [8]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())
'''deaths1 = deaths.groupby(by = 'Edad')
pd.DataFrame(deaths1)'''
# 45 a 49 no, y demás

"deaths1 = deaths.groupby(by = 'Edad')\npd.DataFrame(deaths1)"

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [9]:
# Example 1

dataset = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'cause_name', ['Otras enfermedades isquémicas del corazón'])
'''dataset = mod.row_filter(dataset, 'Total', dataset['Total'] > 0)
dataset.sort_values(by='Total', ascending=False)'''
dataset.sort_values(by='Total', ascending=False)
dataset

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,056 Otras enfermedades isquémicas del corazón,Total,Todas las edades,2015,17837,056,Single cause,Otras enfermedades isquémicas del corazón
1,056 Otras enfermedades isquémicas del corazón,Total,Todas las edades,2017,17369,056,Single cause,Otras enfermedades isquémicas del corazón
2,056 Otras enfermedades isquémicas del corazón,Total,Todas las edades,2016,17148,056,Single cause,Otras enfermedades isquémicas del corazón
3,056 Otras enfermedades isquémicas del corazón,Total,Todas las edades,2012,17107,056,Single cause,Otras enfermedades isquémicas del corazón
4,056 Otras enfermedades isquémicas del corazón,Total,Todas las edades,2013,16877,056,Single cause,Otras enfermedades isquémicas del corazón
...,...,...,...,...,...,...,...,...
853,056 Otras enfermedades isquémicas del corazón,Total,De 00 a 01 año,1981,0,056,Single cause,Otras enfermedades isquémicas del corazón
854,056 Otras enfermedades isquémicas del corazón,Total,De 00 a 01 año,1980,0,056,Single cause,Otras enfermedades isquémicas del corazón
855,056 Otras enfermedades isquémicas del corazón,Total,De 01 a 04 años,2018,0,056,Single cause,Otras enfermedades isquémicas del corazón
856,056 Otras enfermedades isquémicas del corazón,Total,De 01 a 04 años,2017,0,056,Single cause,Otras enfermedades isquémicas del corazón


In [10]:
deaths['cause_name'].unique()


array(['I-XXII.Todas las causas',
       'I.Enfermedades infecciosas y parasitarias',
       'Enfermedades infecciosas intestinales',
       'Tuberculosis y sus efectos tardíos', 'Enfermedad meningocócica',
       'Septicemia', 'Hepatitis vírica', 'SIDA',
       'VIH+ (portador, evidencias de laboratorio del VIH, ...)',
       'Resto de enfermedades infecciosas y parasitarias y sus efectos tardíos',
       'II.Tumores',
       'Tumor maligno del labio, de la cavidad bucal y de la faringe',
       'Tumor maligno del esófago', 'Tumor maligno del estómago',
       'Tumor maligno del colon',
       'Tumor maligno del recto, de la porción rectosigmoide y del ano',
       'Tumor maligno del hígado y vías biliares intrahepáticas',
       'Tumor maligno del páncreas', 'Otros tumores malignos digestivos',
       'Tumor maligno de la laringe',
       'Tumor maligno de la tráquea, de los bronquios y del pulmón',
       'Otros tumores malignos respiratorios e intratorácicos',
       'Tumores malig

In [11]:
# Example 2

group = ['Periodo']
dataset_isquemicas_corazon = mod.groupby_sum(dataset, group)
dataset_isquemicas_corazon = dataset_isquemicas_corazon.sort_values(by='Periodo', ascending=False)
dataset_isquemicas_corazon

,Periodo,Total
7,2018,33262
1,2017,34738
2,2016,34296
0,2015,35674
6,2014,33342
4,2013,33754
3,2012,34214
5,2011,33472
8,2010,33168
9,2009,32354


In [12]:
group1 = ['Edad']
dataset_isquemicas_corazon_edad = mod.groupby_sum(dataset, group1)
dataset_isquemicas_corazon_edad = dataset_isquemicas_corazon_edad.sort_values(by='Edad', ascending=False)
dataset_isquemicas_corazon_edad

,Edad,Total
0,Todas las edades,509696
7,De 95 y más años,25749
4,De 90 a 94 años,66061
1,De 85 a 89 años,103168
2,De 80 a 84 años,102887
3,De 75 a 79 años,76610
5,De 70 a 74 años,51233
6,De 65 a 69 años,32741
8,De 60 a 64 años,20715
9,De 55 a 59 años,13075


In [13]:
# Example 3

datasetCausePeriodo = mod.pivot_table(dataset, 'cause_code', 'Periodo')
datasetCausePeriodo.head()


cause_code,Periodo,056
0,1980,14958
1,1981,15684
2,1982,15742
3,1983,16574
4,1984,16810


## ...and finally, show me some insights with Plotly!!!

In [14]:
'''dataset.drop(dataset[dataset['cause_group'] >= 'Multiple causes'].index, inplace = True)
'''

"dataset.drop(dataset[dataset['cause_group'] >= 'Multiple causes'].index, inplace = True)\n"

In [15]:
# Cufflinks histogram
# Eliminar todas las causas?
dataset_isquemicas_corazon_edad.iplot(kind='bar',
                     title='Otras enfermedades isquémicas del corazón, evolución anual',
                     x = 'Edad',
                     y = 'Total',
                     yTitle='Total',
                     xTitle='Year')


In [16]:
# Cufflinks bar plot
'''
dataset_bar.iplot(kind='bar',
                  x='VARIABLE',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')
'''
dataset_isquemicas_corazon_edad.iplot(kind='line',
             x= 'Edad',
              y= 'Total')

In [17]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

"\ndataset_line.iplot(kind='line',\n                   x='VARIABLE',\n                   xTitle='AXIS TITLE',\n                   yTitle='AXIS TITLE',\n                   title='VIZ TITLE')\n"

In [18]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''

"\ndataset_scatter.iplot(x='VARIABLE', \n                      y='VARIABLE', \n                      categories='VARIABLE',\n                      xTitle='AXIS TITLE', \n                      yTitle='AXIS TITLE',\n                      title='VIZ TITLE')\n"

In [19]:
'''def Edad(df):
    return '''

'def Edad(df):\n    return '

In [20]:
#df[¿?]apply(def)

In [21]:
deaths_all_ages = deaths[deaths['Edad'].str.contains('Todas las edades')]
deaths_all_ages = deaths_all_ages[deaths_all_ages['Sexo'].str.contains('Mujeres|Hombres')]
deaths_all_ages = deaths_all_ages[deaths_all_ages['cause_group'].str.contains('Single cause')]
#deaths_all_ages = deaths_all_ages[deaths_all_ages['cause_name'].str.contains('Enfermedad de Alzheimer')]
#deaths_all_ages = deaths_all_ages[deaths_all_ages['Periodo'].contains(2018)] es datetime
#deaths_all_ages = deaths_all_ages[pd.to_datetime(deaths_all_ages['Periodo'].astype(str)).dt.year == 2017|2018]
deaths_all_ages = deaths_all_ages.sort_values(by='Periodo', ascending=False)
deaths_all_ages

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
6006,001 Enfermedades infecciosas intestinales,Hombres,Todas las edades,2018,415,001,Single cause,Enfermedades infecciosas intestinales
111540,041 Otros tumores de comportamiento incierto ...,Hombres,Todas las edades,2018,1315,041,Single cause,Otros tumores de comportamiento incierto o des...
108966,040 Síndrome mielodisplásico,Hombres,Todas las edades,2018,656,040,Single cause,Síndrome mielodisplásico
107250,039 Tumores benignos,Mujeres,Todas las edades,2018,346,039,Single cause,Tumores benignos
106392,039 Tumores benignos,Hombres,Todas las edades,2018,264,039,Single cause,Tumores benignos
...,...,...,...,...,...,...,...,...
67820,024 Tumor maligno del cuello del útero,Hombres,Todas las edades,1980,0,024,Single cause,Tumor maligno del cuello del útero
248858,084 Malformaciones congénitas del sistema cir...,Mujeres,Todas las edades,1980,472,084,Single cause,Malformaciones congénitas del sistema circulat...
68678,024 Tumor maligno del cuello del útero,Mujeres,Todas las edades,1980,313,024,Single cause,Tumor maligno del cuello del útero
6044,001 Enfermedades infecciosas intestinales,Hombres,Todas las edades,1980,187,001,Single cause,Enfermedades infecciosas intestinales


In [22]:
'''dataset = mod.row_filter(deaths, 'Sexo', ['Total'])
dataset = mod.row_filter(dataset, 'cause_name', ['Otras enfermedades isquémicas del corazón'])
dataset = mod.row_filter(dataset, 'Total', dataset['Total'] > 0)
dataset.sort_values(by='Total', ascending=False)
dataset.sort_values(by='Total', ascending=False)
dataset

deaths_all_ages = deaths[deaths['Edad'].str.contains('Todas las edades')]
deaths_all_ages = deaths_all_ages[deaths_all_ages['Sexo'].str.contains('Mujeres|Hombres')]
deaths_all_ages = deaths_all_ages[deaths_all_ages['cause_group'].str.contains('Single cause')]
#deaths_all_ages = deaths_all_ages[deaths_all_ages['cause_name'].str.contains('Enfermedad de Alzheimer')]
#deaths_all_ages = deaths_all_ages[deaths_all_ages['Periodo'].contains(2018)] es datetime
#deaths_all_ages = deaths_all_ages[pd.to_datetime(deaths_all_ages['Periodo'].astype(str)).dt.year == 2017|2018]
deaths_all_ages = deaths_all_ages.sort_values(by='Periodo', ascending=False)
deaths_all_ages'''

"dataset = mod.row_filter(deaths, 'Sexo', ['Total'])\ndataset = mod.row_filter(dataset, 'cause_name', ['Otras enfermedades isquémicas del corazón'])\ndataset = mod.row_filter(dataset, 'Total', dataset['Total'] > 0)\ndataset.sort_values(by='Total', ascending=False)\ndataset.sort_values(by='Total', ascending=False)\ndataset\n\ndeaths_all_ages = deaths[deaths['Edad'].str.contains('Todas las edades')]\ndeaths_all_ages = deaths_all_ages[deaths_all_ages['Sexo'].str.contains('Mujeres|Hombres')]\ndeaths_all_ages = deaths_all_ages[deaths_all_ages['cause_group'].str.contains('Single cause')]\n#deaths_all_ages = deaths_all_ages[deaths_all_ages['cause_name'].str.contains('Enfermedad de Alzheimer')]\n#deaths_all_ages = deaths_all_ages[deaths_all_ages['Periodo'].contains(2018)] es datetime\n#deaths_all_ages = deaths_all_ages[pd.to_datetime(deaths_all_ages['Periodo'].astype(str)).dt.year == 2017|2018]\ndeaths_all_ages = deaths_all_ages.sort_values(by='Periodo', ascending=False)\ndeaths_all_ages"

In [23]:
#Cuál es la mayor causa de la muerte histórica?
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [24]:
myFigure = px.line(deaths_all_ages, x="Causa de muerte", y="Total",
                 width=1800, height=1000)
 
# Display the figure
myFigure.show()

In [25]:
fig = px.bar(deaths_all_ages, x="Periodo", y="Total", color="Sexo", title="Sexo")
fig.show()

In [26]:
fig_enf = px.bar(deaths_all_ages, x="Periodo", y="Total", 
                 color="Sexo", barmode="group")#, width=2500, height=1000)
    
fig_enf.show()

In [27]:
'''deaths_all_ages.iplot(kind='bar')'''

"deaths_all_ages.iplot(kind='bar')"

In [28]:
fig_enf = px.histogram(deaths_all_ages, x="Periodo", y="Total", 
                 color="Sexo", barmode="group",
                histfunc='avg',
             height=400)
    
fig_enf.show()

In [29]:
deaths_all_agess = deaths[deaths['Edad'].str.contains('Todas las edades')]
deaths_all_agess = deaths_all_agess[deaths_all_agess['Sexo'].str.contains('Mujeres|Hombres')]
deaths_all_agess = deaths_all_agess[deaths_all_agess['cause_group'].str.contains('Single cause')]
deaths_all_agess = deaths_all_agess[deaths_all_agess['cause_name'].str.contains('Trastornos mentales orgánicos, senil y presenil')]
#deaths_all_agess = deaths_all_agess[deaths_all_agess['Periodo'].contains(2018)] es datetime
#deaths_all_agess = deaths_all_agess[pd.to_datetime(deaths_all_agess['Periodo'].astype(str)).dt.year == 2017|2018]
deaths_all_agess = deaths_all_agess.sort_values(by='Periodo', ascending=False)
deaths_all_agess.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
132132,"046 Trastornos mentales orgánicos, senil y pr...",Hombres,Todas las edades,2018,7162,046,Single cause,"Trastornos mentales orgánicos, senil y presenil"
132990,"046 Trastornos mentales orgánicos, senil y pr...",Mujeres,Todas las edades,2018,14507,046,Single cause,"Trastornos mentales orgánicos, senil y presenil"
132991,"046 Trastornos mentales orgánicos, senil y pr...",Mujeres,Todas las edades,2017,14037,046,Single cause,"Trastornos mentales orgánicos, senil y presenil"
132133,"046 Trastornos mentales orgánicos, senil y pr...",Hombres,Todas las edades,2017,6998,046,Single cause,"Trastornos mentales orgánicos, senil y presenil"
132992,"046 Trastornos mentales orgánicos, senil y pr...",Mujeres,Todas las edades,2016,13520,046,Single cause,"Trastornos mentales orgánicos, senil y presenil"


In [30]:
fig_enff = px.bar(deaths_all_agess, x="Periodo", y="Total", 
                 color="Sexo", barmode="group", title = 'Trastornos mentales orgánicos, senil y presenil')#, width=2500, height=1000)
    
fig_enff.show()

In [31]:
deaths_all_ages.iplot(kind='bar',
             x= 'Causa de muerte',
              y= 'Total')

#### FUNCTIONS

#### Group by

groupby_sum(df, group_vars, agg_var='Total', sort_var='Total'):


#### Bins (añadir columna)

#### Linear regression

In [32]:
inicio = mod.row_filter(deaths,'Sexo',['Mujeres', 'Hombres']) # Incluir
inicio = mod.nrow_filter(inicio, 'cause_group',['Multiple causes']) # Excluir
inicio = mod.row_filter(inicio, 'Edad',['Todas las edades']) # Incluir
inicio = inicio.sort_values(by='Total', ascending=True)
inicio = inicio.loc[inicio['Total'] > 7000]
inicio 

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
439,056 Otras enfermedades isquémicas del corazón,Hombres,Todas las edades,1997,7015,056,Single cause,Otras enfermedades isquémicas del corazón
438,067 Otras enfermedades del sistema respiratorio,Hombres,Todas las edades,2003,7019,067,Single cause,Otras enfermedades del sistema respiratorio
437,057 Insuficiencia cardíaca,Hombres,Todas las edades,2007,7032,057,Single cause,Insuficiencia cardíaca
436,058 Otras enfermedades del corazón,Mujeres,Todas las edades,1983,7033,058,Single cause,Otras enfermedades del corazón
435,056 Otras enfermedades isquémicas del corazón,Mujeres,Todas las edades,2014,7034,056,Single cause,Otras enfermedades isquémicas del corazón
...,...,...,...,...,...,...,...,...
4,059 Enfermedades cerebrovasculares,Mujeres,Todas las edades,1980,27171,059,Single cause,Enfermedades cerebrovasculares
3,059 Enfermedades cerebrovasculares,Mujeres,Todas las edades,1985,27519,059,Single cause,Enfermedades cerebrovasculares
2,059 Enfermedades cerebrovasculares,Mujeres,Todas las edades,1984,27562,059,Single cause,Enfermedades cerebrovasculares
1,059 Enfermedades cerebrovasculares,Mujeres,Todas las edades,1983,28002,059,Single cause,Enfermedades cerebrovasculares


In [33]:
fig_enfff = px.bar(inicio, x="Total", y="cause_name", 
                  barmode = 'group', color= 'Sexo', title = 'Causas mayor a 7000 de muertes de hombres y de mujeres desde 1980', 
                   width=1000, height=500)
    
fig_enfff.show()

---

In [63]:
inicio_d = mod.row_filter(deaths,'Sexo',['Mujeres', 'Hombres']) # Incluir
inicio_d = mod.nrow_filter(inicio_d, 'cause_group',['Multiple causes']) # Excluir
inicio_d = mod.row_filter(inicio_d, 'Edad',['Todas las edades']) # Incluir
#inicio_d = mod.row_filter(inicio_d, 'Periodo',['2018','2017']) # Incluir
inicio_d = inicio_d.sort_values(by='Total', ascending=True)
inicio_d = inicio_d.loc[inicio_d['Total'] > 2500]
inicio_d = inicio_d.loc[inicio_d['Periodo'] > 2008]
inicio_d = inicio_d.groupby(['cause_name','Sexo']).sum()

inicio_d = inicio_d.reset_index()
inicio_d = inicio_d.sort_values('Total')
inicio_d

C:\Users\marym\AppData\Local\Temp\ipykernel_16608\3003644261.py:8: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,cause_name,Sexo,Periodo,Total
42,"Tumor maligno de sitios mal definidos, secunda...",Mujeres,2009,2981
32,"Paro cardíaco, muerte sin asistencia y otra ca...",Mujeres,4019,5590
49,"Tumor maligno del recto, de la porción rectosi...",Hombres,6051,7587
31,"Paro cardíaco, muerte sin asistencia y otra ca...",Hombres,8042,13232
24,Otras enfermedades del sistema genitourinario,Mujeres,10080,15511
50,"Tumores malignos del tejido linfático, de los ...",Hombres,12093,15705
33,Suicidio y lesiones autoinfligidas,Hombres,16114,21918
19,Otras enfermedades de los vasos sanguíneos,Hombres,20135,26657
41,"Tumor maligno de sitios mal definidos, secunda...",Hombres,20135,27593
48,Tumor maligno del páncreas,Mujeres,20135,29945


In [59]:
inicio_d.Total.max()

173928

In [64]:
fig_2000 = px.bar(inicio_d, x="Total", y="cause_name", 
                  barmode = 'group', color= 'Sexo', title = 'Causas mayor a 2000 de muertes de hombres y de mujeres de la última década', 
                   width=1000, height=500)
    
fig_2000.show()

---

In [36]:
inicio_group = mod.groupby_sum(inicio, 'cause_name', agg_var='Total', sort_var='Total')
inicio_group

,cause_name,Total
0,Enfermedades cerebrovasculares,1467841
1,Infarto agudo de miocardio,803924
2,Insuficiencia cardíaca,635396
3,"Tumor maligno de la tráquea, de los bronquios ...",565825
4,Otras enfermedades del corazón,471767
5,Enfermedades crónicas de las vías respiratoria...,372223
6,Otras enfermedades isquémicas del corazón,286633
7,Otras enfermedades del sistema respiratorio,259697
8,"Trastornos mentales orgánicos, senil y presenil",211155
9,Enfermedad de Alzheimer,102038


In [37]:
fig_enffff = px.bar(inicio_group, x="cause_name", y="Total", 
                  width=2000, height=1000)
    
fig_enffff.show()

In [38]:
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


Cuantas causas de muerte existen? '102 en 14 grupos'

Distribuición por edades? => Histograma

Cuál es la mayor causa de la muerte histórica?

Cuáles la evolución de muertes por ______?

Cuáles son las principales causas de muerte infantil y su evolución en el tiempo? 

Cuál es la causa de muerte que menos os debe preocupar?

Varias librerías; potly, cuf, sns, matplotlib

ETLs, KPIs